In [1]:
import pandas as pd

feature_names = ['party', 'handicapped-infants','water-project-cost-sharing',
                'adoption-of-the-budget-resolution','physician-fee-freeze',
                'el-salvador-aid','religious-groups-in-schools',
                'anti-satellite-test-ban','aid-to-nicaraguan-contras',
                'mx-missile','immigration','synfuels-corporation-cutback',
                'education-spending','superfund-right-to-sue','crime',
                'duty-free-exports','export-administration-act-sotuh-africa']

voting_data = pd.read_csv('voting-records_house-votes-84.csv', na_values = ['?'],
                         names = feature_names)
voting_data.head()

,party,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-sotuh-africa
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
2,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y


In [2]:
voting_data.describe()

,party,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-sotuh-africa
count,435,423,387,424,424,420,424,421,420,413,428,414,404,410,418,407,331
unique,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
top,democrat,n,y,y,n,y,y,y,y,y,y,n,n,y,y,n,y
freq,267,236,195,253,247,212,272,239,242,207,216,264,233,209,248,233,269


In [3]:
voting_data.dropna(inplace = True)
voting_data.describe()

,party,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-sotuh-africa
count,232,232,232,232,232,232,232,232,232,232,232,232,232,232,232,232,232
unique,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
top,democrat,n,n,y,n,y,y,y,y,n,y,n,n,y,y,n,y
freq,124,136,125,123,119,128,149,124,119,119,128,152,124,127,149,146,189


In [4]:
voting_data.replace(('y','n'),(1,0),inplace = True)
voting_data.replace(('democrat','republican'),(1,0),inplace = True)

In [5]:
all_features = voting_data[feature_names].values
all_classes = voting_data['party'].values

In [19]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split, cross_val_score
X_train, X_test, y_train, y_test = train_test_split(all_features, all_classes, test_size = .1, random_state = 0)
X_train.shape, y_train.shape

((208, 17), (208,))

Now, we create the topology

In [23]:
def create_model():
    model = Sequential()
    model.add(Dense(64, activation = 'relu', input_dim = 16, kernel_initializer='normal')) #relu(x) = max(x,0), 512 neurons
    model.add(Dense(64, activation = 'relu', kernel_initializer='normal'))
    model.add(Dense(1, activation = 'sigmoid', kernel_initializer='normal'))
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam',metrics =['accuracy'])
    return model

from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

estimator = KerasClassifier(build_fn = create_model, epochs = 100, verbose = 0)
cv_scores = cross_val_score(estimator, all_features, all_classes, cv = 10)
cv_scores.mean()

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/wrappers/scikit_learn.py", line 223, in fit
    return super(KerasClassifier, self).fit(x, y, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/wrappers/scikit_learn.py", line 166, in fit
    history = self.model.fit(x, y, **fit_args)
  File "/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py", line 108, in _method_wrapper
    return method(self, *args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.

nan

In [21]:
def create_model():
    model = Sequential()
    model.add(Dense(64, activation = 'relu', input_shape = (17,))) #relu(x) = max(x,0), 512 neurons
    model.add(Dropout(0.5)) #randomly sets input units to 0 with frequency rate to prevent overfitting
    model.add(Dense(64, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation = 'sigmoid'))
    model.compile(loss = 'binary_crossentropy', optimizer = 'rmsprop',metrics =['accuracy'])
    return model

from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

estimator = KerasClassifier(build_fn = create_model, epochs = 100, verbose = 0)
cv_scores = cross_val_score(estimator, all_features, all_classes, cv = 10)
cv_scores.mean()

1.0

In [16]:
model = Sequential()
model.add(Dense(64, activation = 'relu', input_shape = (17,))) #relu(x) = max(x,0), 512 neurons
model.add(Dropout(0.5)) #randomly sets input units to 0 with frequency rate to prevent overfitting
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'rmsprop',metrics =['accuracy'])

Run the model

In [17]:
history = model.fit(X_train, y_train,
                   batch_size = 100,
                   epochs = 10,
                   verbose = 2,
                   validation_data = (X_test, y_test))

Epoch 1/10
3/3 - 0s - loss: 0.7422 - accuracy: 0.4231 - val_loss: 0.6443 - val_accuracy: 0.7500
Epoch 2/10
3/3 - 0s - loss: 0.6566 - accuracy: 0.6250 - val_loss: 0.5867 - val_accuracy: 0.8750
Epoch 3/10
3/3 - 0s - loss: 0.5920 - accuracy: 0.7452 - val_loss: 0.5346 - val_accuracy: 0.9167
Epoch 4/10
3/3 - 0s - loss: 0.5919 - accuracy: 0.7260 - val_loss: 0.5026 - val_accuracy: 0.9167
Epoch 5/10
3/3 - 0s - loss: 0.5376 - accuracy: 0.8125 - val_loss: 0.4691 - val_accuracy: 0.9167
Epoch 6/10
3/3 - 0s - loss: 0.5020 - accuracy: 0.8365 - val_loss: 0.4427 - val_accuracy: 0.9167
Epoch 7/10
3/3 - 0s - loss: 0.4721 - accuracy: 0.8462 - val_loss: 0.4136 - val_accuracy: 0.9583
Epoch 8/10
3/3 - 0s - loss: 0.4667 - accuracy: 0.8606 - val_loss: 0.3834 - val_accuracy: 0.9583
Epoch 9/10
3/3 - 0s - loss: 0.4262 - accuracy: 0.8990 - val_loss: 0.3586 - val_accuracy: 0.9583
Epoch 10/10
3/3 - 0s - loss: 0.4109 - accuracy: 0.8942 - val_loss: 0.3264 - val_accuracy: 0.9583
